In [ ]:
from robot_properties_solo.solo12wrapper import Solo12Config
from robot_properties_solo.solo12wrapper import Solo12Robot
from bullet_utils.env import BulletEnvWithGround
from simulate_solo import Simulator
import conf_solo12_trot as conf

# initialize robot in pybullet
env = BulletEnvWithGround()
simulator = Simulator(env, Solo12Robot(), conf)
robot = Solo12Config.buildRobotWrapper()
q0, dq0 = simulator.q0, simulator.nu0
simulator.robot.reset_state(q0, dq0) 
simulator.robot.pin_robot.framesForwardKinematics(q0)

In [ ]:
# import conf_solo12_trot as conf
# import matplotlib.pylab as plt 
# from alternating_planner import ComboPlanner
# import pinocchio as pin
# import sys

# planner = ComboPlanner(conf, MPC=True)
# sol = planner.run_OL_MPC()
# if conf.WITH_MESHCAT_DISPLAY:
#     viz = pin.visualize.MeshcatVisualizer(
#     robot.model, robot.collision_model, robot.visual_model)
#     try:
#         viz.initViewer(open=True)
#     except ImportError as err:
#         print(
#             "Error while initializing the viewer. Make sure you installed Python meshcat"
#         )
#         print(err)
#         sys.exit(0)
#     viz.loadViewerModel()
#     for sol_k in sol:
#         q = sol_k['jointPos'][0]        
#         viz.display(q)


In [ ]:
import conf_solo12_trot as conf
import matplotlib.pylab as plt 
from alternating_planner import ComboPlanner
import pinocchio as pin 
import numpy as np 
import time 

planner = ComboPlanner(conf, MPC=True)
N_traj, N_mpc_centroidal, N_mpc_wbd = planner.N_traj, planner.N_mpc_centroidal, planner.N_mpc_wbd
rmodel, rdata = simulator.robot.pin_robot.model, simulator.robot.pin_robot.data
N_inner = int(conf.dt/conf.dt_ctrl)
centroidal_planner = planner.centroidal_planner
wbd_planner = planner.wbd_planner
hg0 = centroidal_planner.x_init[0]
# create open-loop solution tuples
wbd_nx, wbd_nu = len(wbd_planner.x_init[0]), len(wbd_planner.u_init[0])
centroidal_nx, centroidal_nu = 9, 12
X_sim_centroidal = np.zeros((N_traj, N_mpc_centroidal+1, wbd_nx))
U_sim_centroidal = np.zeros((N_traj, N_mpc_centroidal, wbd_nu))
wbd_sol = []
# create closed-loop solution tuples
X_sol_centroidal = np.zeros((N_traj, centroidal_nx))
U_sol_centroidal = np.zeros((N_traj, centroidal_nu))
Kp = 2.5*np.eye(12)
Kd = 0.01*np.eye(12)
for traj_time_idx in range(N_traj):
    centroidal_planner.update_ocp(traj_time_idx, hg0)
    if centroidal_planner.ocp.solver_options.nlp_solver_type == 'SQP_RTI':
        # feedback rti_phase (solving QP)
        print('starting RTI feedback phase ' + '...')
        centroidal_planner.acados_solver.options_set('rti_phase', 2)
        t_feedback = time.time()
        status = centroidal_planner.acados_solver.solve()
        elapsed_feedback = time.time() - t_feedback
        print('RTI feedback phase took ' + str(elapsed_feedback) + " seconds")
        centroidal_planner.acados_solver.print_statistics()
        if status == 0:
            print("HOORAY ! found a solution after :", 
                    centroidal_planner.elapsed_prep+elapsed_feedback, " seconds")
        else:
            raise Exception(f'acados returned status {status}.')
    else:
        t = time.time()
        status = centroidal_planner.acados_solver.solve()
        elapsed_time= time.time() - t
        centroidal_planner.acados_solver.print_statistics()
        if status == 0:
            print("HOORAY ! found a solution after :", elapsed_time, " seconds")
        else:
            raise Exception(f'acados returned status {status}.')        
    x_sol = np.array([centroidal_planner.acados_solver.get(i,"x") for i in range(N_mpc_centroidal+1)])
    u_sol = np.array([centroidal_planner.acados_solver.get(i,"u") for i in range(N_mpc_centroidal)])
    # add WBD tracking costs from the centroidal solver solution
    wbd_planner.update_ocp(traj_time_idx, centroidalTask=x_sol, forceTask=u_sol)
    # solve WBD OCP
    if traj_time_idx == 0:
        wbd_planner.solver.solve(wbd_planner.x_init, wbd_planner.u_init)  
    else:
        wbd_planner.solver.solve(xs, us)
    xs = [wbd_planner.solver.xs[i] for i in range(len(wbd_planner.solver.xs))]
    us = [wbd_planner.solver.us[i] for i in range(len(wbd_planner.solver.us))]
    # save open-loop solution
    sol_k = wbd_planner.get_solution_trajectories()
    # scaling DDP gains (what is the proper way of doing this?)
    gains = sol_k['gains'][0]*(conf.dt/N_inner)
    q_des, qdot_des, tau_ff = wbd_planner.interpolate_one_step(
                        q=sol_k['jointPos'][0], q_next=sol_k['jointPos'][1],
                   qdot=sol_k['jointVel'][0],qdot_next=sol_k['jointVel'][1],
                    tau=sol_k['jointTorques'][0], tau_next=sol_k['jointTorques'][1]
                        )
    for ctrl_time_idx in range(N_inner):
        q_k, dq_k = simulator.robot.get_state()
        Kp = gains[:, 6:18]
        Kd = gains[:, 24:]
        tau_k = tau_ff[ctrl_time_idx] + \
                  Kp @ (q_des[ctrl_time_idx, 7:] - q_k[7:]) + \
                  Kd @ (q_des[ctrl_time_idx, 7:] - dq_k[6:])
        simulator.robot.send_joint_command(tau_k)
        # step simulation 
        simulator.env.step(sleep=True)     
    q_k, dq_k = simulator.robot.get_state()
    simulator.robot.pin_robot.framesForwardKinematics(q_k)
    com = pin.centerOfMass(rmodel, rdata, q_k, dq_k)
    simulator.robot.pin_robot.centroidalMomentum(q_k, dq_k)
    wbd_sol += [sol_k]
    # # save closed-loop solution
    X_sol_centroidal[traj_time_idx] = x_sol[0]
    U_sol_centroidal[traj_time_idx] = u_sol[0]
    # warm-start solver from the previous solution 
    xs = xs[1:] + [xs[-1]]     
    us = us[1:] + [us[-1]]    
    # update solvers initial conditions
    # OL-MPC
    # hg0 = x_sol[1]
    # x0 = xs[0]
    # CL-MPC
    hg0 =  np.concatenate([com, np.array(rdata.hg)])
    x0 = np.concatenate([q_k, dq_k])
    wbd_planner.x0 = x0